In [28]:
import importlib

import wrds
import pandas as pd
import datetime
import numpy as np
import pickle
from time import time, sleep
import argparse as ap
import sys
import csv
from dateutil.relativedelta import relativedelta
from collections import OrderedDict
from fredapi import Fred

db = wrds.Connection()

Enter your WRDS username [dlflath23]:dflath19
Enter your password:········
WRDS recommends setting up a .pgpass file.
You can find more info here:
https://www.postgresql.org/docs/9.5/static/libpq-pgpass.html.
Loading library list...
Done


In [155]:
def option_table(date):
    s = 'opprcd' + date[0:4]
    return s

def one_sec_option_table(secid,date):
    q1 = ("select secid, date, symbol, optionid, exdate, strike_price, best_bid, best_offer "
      "from optionm.%s "
      "where cp_flag = 'P' and secid = %g " % (option_table(date), secid) )
    df = db.raw_sql(q1)
    return df

def adj_date(df, date):
    df = df.loc[df['date'] >= datetime.datetime.strptime(date, "%Y-%m-%d").date()]
    return df.date.unique()[0].strftime('%Y-%m-%d')

def get_secid(ticker, date):
    q1 = ("select secid, optionid, ticker, effect_date "
      "from optionm.optionmnames "
      "where ticker = '%s' and effect_date < '%s'" % (ticker, date) )
    df = db.raw_sql(q1)
    return df.sort_values(by=['effect_date']).iloc[-1].secid

def target_ex_date(start_date, days):
    date_1 = datetime.datetime.strptime(start_date, "%Y-%m-%d")
    end_date = date_1 + datetime.timedelta(days=days)
    #date_string = end_date.strftime('%Y-%m-%d')
    return end_date.date()

def available_ex_dates(df,date):
    df = df.loc[df['date'] == datetime.datetime.strptime(date, "%Y-%m-%d").date()]
    return df.exdate.unique()

def available_strikes(df,date,ex_date):
    df = df.loc[df['date'] == datetime.datetime.strptime(date, "%Y-%m-%d").date()]
    df = df.loc[df['exdate'] == ex_date]
    strikes = df.strike_price.unique()
    return [x / 1000.00 for x in strikes]

def nearest(list, pivot):
    return min(list, key=lambda x: abs(x - pivot))

def one_day_pricing(df,date,ex_date):
    df = df.loc[df['date'] == datetime.datetime.strptime(date, "%Y-%m-%d").date()]
    df = df.loc[df['exdate'] == ex_date]
    return df

def one_contract_pricing(df,date,ex_date,strike_price):
    df = df.loc[df['date'] >= datetime.datetime.strptime(date, "%Y-%m-%d").date()]
    df = df.loc[df['exdate'] == ex_date]
    df = df.loc[df['strike_price'] == strike_price]
    return df

def hi_lo_avg(secid,date):
    q1 = ("select secid, date, high, low, close "
      "from optionm.secprd "
      "where secid = %g and date <= '%s'" % (secid, date) )
    df = db.raw_sql(q1)
    df['sma'] = df['close'].rolling(200).mean()
    hl = round((list(df.high)[-1] + list(df.low)[-1])/2.00,2)
    if hl > list(df.sma)[-1]:
        #hl = round(list(df.sma)[-1],2)
        hl = hl
    else:
        hl = hl
    return hl

def nearest_strike(p,delta,strikes,verbose):
    p_target = np.float64(p * delta)
    s = nearest(strikes,p_target)
    buffer_level = p * (delta + (1.00-delta)/2.00)
    if verbose:
        print("Buffet level: " + str(buffer_level))
    for i in xrange(20):
        if s > buffer_level:
            p_target = p_target * (1+(-0.01 * i))
            if verbose:
                print("P Target adj: " + str(p_target))
            s = nearest(strikes,p_target)
            if verbose:
                print("Nearest: " + str(s))
        else:
            break
    if s > p:
        return None
    else:
        return s * 1000.00

def nearest_end_date(end_dates, end_date_date):
    date2 = end_date_date
    while True:
        date = nearest(end_dates,date2)
        if min(end_dates) > end_date_date:
            date = date2
            break
        if date <= end_date_date:
            break
        date2 = date2 - datetime.timedelta(days=1)
    return date

def average_ex_date(results):
    d=[]
    for t in results:
        d.append(results[t]['expiry']) 
    a = np.array(list(d), dtype='datetime64').view('i8').mean().astype('datetime64[D]')
    return pd.to_datetime(str(a))

def cost_final(ticker,start_date,end_date,delta,fwd,verbose=False):
    sec_id = get_secid(ticker, start_date)
    df = one_sec_option_table(sec_id,start_date)
    if verbose:
        print("Security ID: " + str(sec_id))
    start_date = adj_date(df,start_date)
    if verbose:
        print("Adjusted Start Date: " + start_date)
    dates = available_ex_dates(df, start_date)
    if verbose:
        print("Available ex dates: ")
        print(dates)
    target_ex = target_ex_date(start_date,fwd)
    if verbose:
        print("Target ex date: " + target_ex.strftime('%Y-%m-%d'))
    nearest_ex = nearest(dates,target_ex)
    if verbose:
        print("Nearest ex date: " + nearest_ex.strftime('%Y-%m-%d'))
    strikes = available_strikes(df, start_date,nearest_ex)
    if verbose:
        print("Available strikes: ")
        print(strikes)
    hl = hi_lo_avg(sec_id,start_date)
    if verbose:
        print("Current price: " + str(hl)) 
        print("Target strike: " + str(round((hl * delta),2)))  
    strike = nearest_strike(hl,delta,strikes,verbose)
    if verbose:
        print("Closest strike: " + str(strike/1000.00))
        print('.............................')
        print('.............................')
    prices = one_contract_pricing(df,start_date,nearest_ex,strike)
    start_bid = prices.iloc[0].best_bid
    start_ask = prices.iloc[0].best_offer
    cost = round((start_bid * 0.67) + (start_ask * 0.33),3)
    cost_date = prices.iloc[0].date
    if end_date is None:
        final_bid = prices.iloc[-1].best_bid
        final_ask = prices.iloc[-1].best_offer
        final = round((final_bid * 0.33) + (final_ask * 0.67),3)
        final_date = nearest_ex.strftime('%Y-%m-%d')
    else:
        end_dates = prices.date.unique()
        end_date_date = datetime.datetime.strptime(end_date, "%Y-%m-%d").date()
        closest_end_date = nearest_end_date(end_dates,end_date_date)
        prices_data = prices.loc[prices['date'] == closest_end_date]
        final_bid = prices_data.best_bid.iloc[0]
        final_ask = prices_data.best_offer.iloc[0]
        final = round((final_bid * 0.33) + (final_ask * 0.67),3)
        final_date = prices_data.date.iloc[0]
    s_yield = round(cost / (strike/1000.00*1.00),4)
    hp = (nearest_ex - cost_date).days*1.00/360.00
    a_yield = round(((1.00 + s_yield)**(1.00/hp)) - 1.00,4)
    d = OrderedDict()
    d["ticker"] = ticker
    d["secid"] = sec_id
    d["trade_date"] = cost_date.strftime('%Y-%m-%d')
    d["expiry"] = nearest_ex.strftime('%Y-%m-%d')
    d["current"] = hl
    d["strike"] = strike/1000.00
    d["delta"] = round(strike/1000.00/hl - 1.00,4)
    d["c_date"] = cost_date.strftime('%Y-%m-%d')
    d["c_bid"] = start_bid
    d["c_ask"] = start_ask
    d["c_mid"] = cost
    d["f_date"] = str(final_date)
    d["f_bid"] = final_bid
    d["f_ask"] = final_ask
    d["f_mid"] = final
    d["gross_profit"] = (cost - final)
    d["s_yield"] = s_yield
    d["a_yield"] = a_yield
 
    return d

def calculate_number_of_contracts(starting_capital,results,max_pos_size):
    number_of_contracts = len(results.values())
    target_dollar_per_position = starting_capital / number_of_contracts
    for r in results.values():
        if target_dollar_per_position > (starting_capital * max_pos_size):  
            r["target_notional"] = (starting_capital * max_pos_size)
            contracts = round(starting_capital * max_pos_size/r["strike"]/100,0)
        else:
            r["target_notional"] = round(target_dollar_per_position,2)
            contracts = round(target_dollar_per_position/r["strike"]/100,0)
        if contracts < 1:
            r["contracts"] = 1
        else:
            r["contracts"] = contracts
        r["actual_notional"] = round(r["contracts"] * r["strike"] * 100, 2)
        r["total_profit"] = round(r["contracts"] * r["gross_profit"] * 100,2)
        r["ending_total"] = r["target_notional"] + r["total_profit"] 
    return results

def filter_for_min_yield(results,min_yield):
    remove = []
    for r in results:
        yld = results[r]["a_yield"]
        if yld < min_yield:
            #results.pop(r)
            remove.append(r)
    for p in remove:
        results.pop(p)
    return results

def filter_yield_rank(results,n,outlier_break):

    ranks = []
    for r in results:
        yld = results[r]["a_yield"]
        ranks.append((r,yld))
    sorted_by_second = sorted(ranks, key=lambda tup: tup[1])
    remove = []
    l = len(sorted_by_second)
    break_index = int(round(l * outlier_break,0))
    if l > n:
        for t in sorted_by_second[(l-break_index):]:
            remove.append(t[0])
        for t in sorted_by_second[:(l-n-break_index)]:
            remove.append(t[0])
        for p in remove:
            results.pop(p)
    else:
        None
    return results

def total_period_profit(results):
    total = 0.00 
    for r in results.values():
        total = total + r["total_profit"]
    return round(total,2)

def get_fred_data():
    fred = Fred(api_key='5efb16a4685c34b5a642fa2f07ad97b7')
    d = fred.get_series('DTB3')
    df = pd.DataFrame(data=d).ffill()
    return df

fred_data = get_fred_data()

def add_interest(start_date,starting_capital,results,fred_data):
    start_date = np.datetime64(datetime.datetime.strptime(start_date, "%Y-%m-%d"))
    dates = fred_data.index.values
    date = nearest(dates,start_date)
    #print("Closest date: " + str(date)[:10])
    #print("Rate raw: " + str(fred_data.loc[date][0]))
    rate = fred_data.loc[date][0] / 100.00
    through_date = np.datetime64(average_ex_date(results))
    holding_period = (through_date - start_date).item().days*1.00/360.00
    interest_earned = starting_capital * rate * holding_period
    #print("Interest rate: " + str(round(rate,4)*100) + "%")
    #print("Interest earned: " + "$" + str(round(interest_earned,2)))
    return round(interest_earned,2)

def pretty_print(cf_data):
    print("Trade date:" + cf_data["trade_date"])
    print("Expiry date: " + cf_data["expiry"])
    print("Current Price: " + str(cf_data["current"]))
    print("Strike: " + str(cf_data["strike"]))
    print("Actual delta: " + str(cf_data["delta"] * 100) + "%")
    print("Start Date: " + cf_data["c_date"])
    print("Start Cost: " + str(cf_data["c_bid"]))
    print("Final Date: " + cf_data["f_date"])
    print("Final Bid: " + str(cf_data["f_bid"]))
    print("Final Ask: " + str(cf_data["f_ask"]))
    print("Final Mid: " + str(cf_data["f_mid"]))
    print("Simple Yield: " + str(cf_data["s_yield"] * 100) + "%")
    print("Annualized Yield: " + str(cf_data["a_yield"] * 100) + "%")
    
def confirm_bid_available(results,cf,start_date,start_date_stick):
    if cf["c_bid"] == 0.0:
        while True:
            start_date = datetime.datetime.strptime(start_date, "%Y-%m-%d")
            start_date = start_date + datetime.timedelta(days=1)
            away = start_date - start_date_stick
            if away.days >= 10:
                print(" -> No available trade dates within 10 days")
                break
            start_date = start_date.strftime('%Y-%m-%d')
            cf = cost_final(ticker,start_date,None,delta,fwd,verbose)
            ed = datetime.datetime.strptime(cf['expiry'], "%Y-%m-%d")
            if cf["c_bid"] > 0.0:
                if (datetime.datetime.strptime(cf['trade_date'], "%Y-%m-%d") - start_date).days > 10:
                    print(" -> Pass, trade date more than 10 days away")
                    break
                else:
                    final_cf = cf
                    break
    else:
        final_cf = cf
    return final_cf

import os
import json

def check_directory(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)
        
def write_csv(start_date,results,start_time):
    print("      --> Done. Writing CSV file")
    directory = '/home/dlflath23/options/results/'+start_time
    check_directory(directory)
    path = os.path.join(directory, 'option_results_' + start_date + '.csv')
    with open(path, 'w') as csvfile:
        fieldnames = results.values()[0].keys()
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for t in results:
            ticker = t
            data = results[t]
            writer.writerow(data)

def write_total_csv(total_pl,start_time):
    directory = '/home/dlflath23/options/results/'+start_time
    check_directory(directory)
    path = os.path.join(directory, 'total_pl_' + start_time + '.csv')
    with open(path, 'w') as csvfile:
        w = csv.writer(csvfile)
        w.writerow(total_pl[0].keys())
        for r in total_pl:
            w.writerow(r.values())

def write_params_file(params,start_time):
    directory = '/home/dlflath23/options/results/'+start_time
    check_directory(directory)
    path = os.path.join(directory, 'params_' + start_time + '.txt')
    text = json.dumps(params)
    f = open(path,'w')
    f.write(text)
    f.close()

In [ ]:
def testing():
    ticker = 'IC'
    start_date = '1996-01-04'
    end_date = None
    start_date_stick = datetime.datetime.strptime(start_date, "%Y-%m-%d")
    delta = 0.95
    fwd = 20
    verbose = True

    cf = cost_final(ticker,start_date,end_date,delta,fwd,verbose)
    return cf
    #pretty_print(cf)
    if cf["c_bid"] == 0.0:
        print('.............................')
        print("No bid on this date, trying next date")
        print('.............................')
        while True:
            start_date = datetime.datetime.strptime(start_date, "%Y-%m-%d")
            start_date = start_date + datetime.timedelta(days=1)
            away = start_date - start_date_stick
            print("Looking ahead " + str(away.days) + " day(s)")
            if away.days >= 10:
                print("No bids within 10 days of target")
                break
            start_date = start_date.strftime('%Y-%m-%d')
            cf = cost_final(ticker,start_date,end_date,delta,fwd,verbose)
            if cf["c_bid"] > 0.0:
                return cf
                #pretty_print(cf)
                break

testing()

In [ ]:
#db.list_tables('optionm')

In [ ]:
#db.describe_table('compm', 'company')

In [159]:
def previous_quarter(date_string):
    ref = datetime.datetime.strptime(date_string, "%Y-%m-%d").date()
    if ref.month < 4:
        return datetime.date(ref.year - 1, 12, 31)
    elif ref.month < 7:
        return datetime.date(ref.year, 3, 31)
    elif ref.month < 10:
        return datetime.date(ref.year, 6, 30)
    return datetime.date(ref.year, 9, 30)

def top_n_mkt_cap_gvkeys(date_string,n,gvkeys):
    date1 = previous_quarter(date_string)
    date2 = previous_quarter(date1.strftime('%Y-%m-%d'))
    gvkeys_list = tuple(["'%s'"%str(i) for i in gvkeys])
    gvkeys_list = ",".join(gvkeys_list)
    q1 = ("select datadate, gvkey,cshoq,prccq,cshoq*prccq as market_cap, curcdq "
            "from compm.fundq "
           "where cshoq>0 and prccq>0 and curcdq='USD' and datadate > '%s' and datadate <= '%s' and gvkey in (%s) "
         "order by market_cap desc "
        "limit %i") % (date2,date1,gvkeys_list,n)
    df = db.raw_sql(q1)
    return list(df.gvkey.unique())

def top_n_mkt_cap_tics(date_string,n,gvkeys):
    date1 = previous_quarter(date_string)
    date2 = previous_quarter(date1.strftime('%Y-%m-%d'))
    gvkeys_list = tuple(["'%s'"%str(i) for i in gvkeys])
    gvkeys_list = ",".join(gvkeys_list)
    q1 = ("select datadate, gvkey,tic,cshoq,prccq,cshoq*prccq as market_cap, curcdq "
            "from compm.fundq "
           "where cshoq>0 and prccq>0 and curcdq='USD' and datadate > '%s' and datadate <= '%s' and gvkey in (%s) "
         "order by market_cap desc "
        "limit %i") % (date2,date1,gvkeys_list,n)
    df = db.raw_sql(q1)
    tics = [t.split('.')[0] for t in list(df.tic.unique())]
    return tics

def tickers_to_secids(tickers,date):
    ticker_list = tuple(["'%s'"%str(i) for i in tickers])
    ticker_list = ",".join(ticker_list)
    q1 = ("select secid, ticker, effect_date "
      "from optionm.optionmnames "
      "where ticker in (%s) and effect_date < '%s'" % (ticker_list, date) )
    df = db.raw_sql(q1)
    df = df.groupby('ticker').apply(lambda g: g[g['effect_date'] == g['effect_date'].max()])
    return list(df.secid.unique())

def secids_to_tickers(secids,date):
    secid_list = tuple(["'%s'"%str(i) for i in secids])
    secid_list = ",".join(secid_list)
    q1 = ("select secid, ticker, effect_date "
      "from optionm.optionmnames "
      "where secid in (%s) and effect_date < '%s'" % (secid_list, date) )
    df = db.raw_sql(q1)
    df = df.groupby('ticker').apply(lambda g: g[g['effect_date'] == g['effect_date'].max()])
    return list(df.ticker.unique())

def gvkeys_for_filtered_security_type(date_string):
    year = date_string[0:4]
    date1 = previous_quarter(date_string)
    date2 = previous_quarter(date1.strftime('%Y-%m-%d'))
    q1 = ("select gvkey,cyear,tpci,exchg,datadate "
            "from compm.secm "
           "where datadate > '%s' and datadate <= '%s' and tpci in ('0','F') and exchg in (11,12,14,19) ")% (date2,date1)
    df = db.raw_sql(q1)
    return list(df.gvkey.unique())

def gvkeys_for_filtered_sectors(date_string,gvkeys):
    gvkeys_list = tuple(["'%s'"%str(i) for i in gvkeys])
    gvkeys_list = ",".join(gvkeys_list)
    q1 = ("select gvkey,gsector,gind,loc "
            "from compm.company "
           "where gsector != '40' and gind not in ('151040','101020','352010') and gvkey in (%s) and loc='USA' ")% gvkeys_list
    df = db.raw_sql(q1)
    return list(df.gvkey.unique())

def get_latest_prices(date_string,gvkeys):
    gvkeys_list = tuple(["'%s'"%str(i) for i in gvkeys])
    gvkeys_list = ",".join(gvkeys_list)
    q1 = ("select datadate,gvkey,prccd,tpci "
            "from compm.secd "
           "where datadate = '%s' and gvkey in (%s) and tpci in ('0','F') ") % (date_string,gvkeys_list)
    df = db.raw_sql(q1)
    while True:
        if df.gvkey.nunique() == 0:
            date = datetime.datetime.strptime(date_string, "%Y-%m-%d") - datetime.timedelta(days=1)
            date_string = date.strftime('%Y-%m-%d')
            q1 = ("select datadate,gvkey,prccd,tpci "
            "from compm.secd "
           "where datadate = '%s' and gvkey in (%s) and tpci in ('0','F') ") % (date_string,gvkeys_list)
            df = db.raw_sql(q1)
        else:
            break
    return df

def get_cheapest_n_tics(date_string,gvkeys,n):
    date1 = previous_quarter(date_string)
    date2 = previous_quarter(date1.strftime('%Y-%m-%d'))
    date3 = date2 - relativedelta(years=4)
    gvkeys_list = tuple(["'%s'"%str(i) for i in gvkeys])
    gvkeys_list = ",".join(gvkeys_list)
    q1 = ("select datadate,gvkey,tic,cusip,oiadpq,cshoq,prccq,pstkq, dlcq, dlttq, cheq,niq,ceqq "
            "from compm.fundq "
           "where datadate <= '%s' and datadate > '%s' and gvkey in (%s) ") % (date2,date3,gvkeys_list)
    df = db.raw_sql(q1)
    df["date"] = pd.to_datetime(df['datadate'], format='%Y-%m-%d')
    df.sort_values(by=["date"])
    df = df.groupby('gvkey').ffill()
    df['oiadpq_ttm'] = df.groupby('gvkey')['oiadpq'].apply(lambda x: x.ffill().rolling(window=4).sum())
    df['ni_ttm'] = df.groupby('gvkey')['niq'].apply(lambda x: x.ffill().rolling(window=4).sum())
    df['eqity_ttm'] = df.groupby('gvkey')['ceqq'].apply(lambda x: x.ffill().rolling(window=4).sum()/4)
    df['roe'] = df['ni_ttm'] / df['eqity_ttm']
    df['roe_3yr_avg'] = df.groupby('gvkey')['roe'].apply(lambda x: x.ffill().rolling(window=12).sum()/12)
    latest_prices = get_latest_prices(date_string,gvkeys)
    df = pd.merge(df, latest_prices, on = 'gvkey', how='left')
    df["market_cap"] = df["cshoq"] * df["prccd"]
    df["ent_value"] = df["market_cap"] + df["pstkq"] + df["dlcq"] + df["dlttq"] - df["cheq"]
    df["ev_ebit"] = (df["ent_value"] / df["oiadpq_ttm"])
    df["net_debt"] = df["dlcq"] + df["dlttq"] - df["cheq"]
    df["net_debt_ebit_ttm"] = df["net_debt"] / df['oiadpq_ttm']
    df = df.round({'market_cap': 2, 'ent_value': 2, 'ev_ebit' : 2, 'roe' : 4, 'roe_3yr_avg' : 4, 'net_debt_ebit_ttm' : 2})
    df = df.loc[df['ev_ebit'] > 0]
    df = df.sort_values(by=["roe_3yr_avg"],ascending=False)
    df = df.groupby('gvkey').apply(lambda g: g[g['date'] == g['date'].max()])
    df = df.loc[df['roe_3yr_avg'] >= 0.15]
    df = df.loc[df['net_debt_ebit_ttm'] <= 3.00]
    df = df.sort_values(by=["ev_ebit"])
    df = df.head(n)
    tics = [t.split('.')[0] for t in list(df.tic.unique())]
    
    #return df.filter(items=['tic','date','niq','ceqq','roe','roe_3yr_avg','ev_ebit','net_debt_ebit_ttm'])
    #return df
    return tics

def get_tics(date_string,n):
    gvkeys = gvkeys_for_filtered_security_type(date_string)
    sector_filtered = gvkeys_for_filtered_sectors(date_string,gvkeys)
    top_n_gvkeys = top_n_mkt_cap_gvkeys(date_string,3000,sector_filtered)
    d = get_cheapest_n_tics(date_string,top_n_gvkeys,n)
    return d

def top_n_impl_vol_tics(date_string,n):
    tics = get_tics(date_string,100)
    secids = tickers_to_secids(tics,date_string)
    secid_list = tuple(["'%s'"%str(i) for i in secids])
    secid_list = ",".join(secid_list)
    s = 'stdopd' + date_string[0:4]
    q1 = ("select date,cp_flag,secid,impl_volatility,cp_flag,days "
                "from optionm.%s "
               "where secid in (%s) and date >= '%s' and cp_flag='P' and days=30 "
         "order by impl_volatility desc ") % (s,secid_list,date_string)
    df = db.raw_sql(q1)
    dates = list(df.date.unique())
    adj_date = nearest(dates, datetime.datetime.strptime(date_string, "%Y-%m-%d").date())
    df = df.loc[df['date'] == adj_date].head(n)
    top_n_secids = list(df.secid.unique())
    tics = secids_to_tickers(top_n_secids,date_string)
    return tics

#top_n_impl_vol_tics('2005-01-01',50)
#get_tics('2005-01-01',50)

In [160]:
import matplotlib.pyplot as plt

start_date = '1996-01-01'
stop_date = '2015-12-31'
start_time = datetime.datetime.strftime(datetime.datetime.now(),"%Y%m%d-%H%M%S")

params = {
'delta' : 0.90,
'fwd' : 20,
'verbose' : False,
'starting_capital' : 1000000,
'max_size' : 0.05,
'min_yield' : 0.05,
'target_n' : 100,
'target_n_top_yields' : 100,
'target_n_top_vol' : 10,
'outlier_break' : 0.00
}

write_params_file(params,start_time)

total_pl=[]
d = OrderedDict()
d['end_date'] = start_date
d['market_value'] = params['starting_capital']
total_pl.append(d.copy())

while True:
    if datetime.datetime.strptime(start_date, "%Y-%m-%d") > datetime.datetime.strptime(stop_date, "%Y-%m-%d"):
        break
    print("Getting tickers for " + start_date)
    tics = get_tics(start_date,params['target_n'])
    #tics = top_n_impl_vol_tics(start_date,params['target_n_top_vol'])
    if params['verbose']:
        print(tics)
    results = {}
    count = 0.0
    print("   Calculating ")
    for t in tics:
        count = count + 1
        i = (count/len(tics))*100
        sys.stdout.write("\r%d%%" % i)
        sys.stdout.flush() 
        start_date_stick = datetime.datetime.strptime(start_date,"%Y-%m-%d")
        try:
            if params['verbose']:
                print("Calculating " + str(t))
            cf = cost_final(t,start_date,None,params['delta'],params['fwd'],params['verbose'])
            if (datetime.datetime.strptime(cf['trade_date'], "%Y-%m-%d") - start_date_stick).days > 10:
                if params['verbose']:
                    print(" -> Pass, trade date more than 10 days away")
                continue
            if cf['secid'] in [o['secid'] for o in results.values()]:
                if params['verbose']:
                    print(" -> Pass, sec id already exists")
                continue
            results[t] = confirm_bid_available(results,cf,start_date,start_date_stick)
        except:
            if params['verbose']:
                print(" -> Ticker " + str(t) + " wasn't able to retrieve data")
            pass

    results = filter_for_min_yield(results,params['min_yield'])
    
    results = filter_yield_rank(results,params['target_n_top_yields'],params['outlier_break'])
    
    if len(results.values()) > 1:  
        results = calculate_number_of_contracts(params['starting_capital'],results,params['max_size'])
        period_profit = total_period_profit(results)
        period_int = add_interest(start_date,params['starting_capital'],results,fred_data)
        print("")
        print("   Starting Capital: " + "$" + str(round(params['starting_capital'],2)))
        print("      Profit from options: " + "$" + str(period_profit))
        print("      Profit from interest: " + "$" + str(period_int))
        params['starting_capital'] = params['starting_capital'] + period_profit + period_int
        print("   Ending Capital: " + "$" + str(round(params['starting_capital'],2)))
        write_csv(start_date,results,start_time)
        start_date = (average_ex_date(results) + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
        d['end_date'] = start_date
        d['market_value'] = params['starting_capital']
        total_pl.append(d.copy())
        write_total_csv(total_pl,start_time)
    else:
        print("")
        start_date = (datetime.datetime.strptime(start_date, "%Y-%m-%d") + datetime.timedelta(days=1)).strftime('%Y-%m-%d')

Getting tickers for 1996-01-01
   Calculating 
100%
Getting tickers for 1996-01-02
   Calculating 
100%
   Starting Capital: $1000000.0
      Profit from options: $-221.6
      Profit from interest: $2485.0
   Ending Capital: $1002263.4
      --> Done. Writing CSV file
Getting tickers for 1996-01-21
   Calculating 
100%
   Starting Capital: $1002263.4
      Profit from options: $5761.9
      Profit from interest: $3743.45
   Ending Capital: $1011768.75
      --> Done. Writing CSV file
Getting tickers for 1996-02-18
   Calculating 
100%
   Starting Capital: $1011768.75
      Profit from options: $-3837.0
      Profit from interest: $3612.01
   Ending Capital: $1011543.76
      --> Done. Writing CSV file
Getting tickers for 1996-03-17
   Calculating 
100%
   Starting Capital: $1011543.76
      Profit from options: $3312.9
      Profit from interest: $4805.39
   Ending Capital: $1019662.05
      --> Done. Writing CSV file
Getting tickers for 1996-04-21
   Calculating 
100%
   Starting Cap

   Calculating 
100%
   Starting Capital: $1060632.04
      Profit from options: $988.6
      Profit from interest: $4337.4
   Ending Capital: $1065958.04
      --> Done. Writing CSV file
Getting tickers for 1999-02-21
   Calculating 
100%
   Starting Capital: $1065958.04
      Profit from options: $-23238.9
      Profit from interest: $3597.61
   Ending Capital: $1046316.75
      --> Done. Writing CSV file
Getting tickers for 1999-03-21
   Calculating 
100%
   Starting Capital: $1046316.75
      Profit from options: $6064.6
      Profit from interest: $3452.85
   Ending Capital: $1055834.2
      --> Done. Writing CSV file
Getting tickers for 1999-04-18
   Calculating 
100%
   Starting Capital: $1055834.2
      Profit from options: $4914.0
      Profit from interest: $4208.09
   Ending Capital: $1064956.29
      --> Done. Writing CSV file
Getting tickers for 1999-05-23
   Calculating 
100%
   Starting Capital: $1064956.29
      Profit from options: $2269.9
      Profit from interest: $

100%
   Starting Capital: $1174837.71
      Profit from options: $5978.7
      Profit from interest: $1506.73
   Ending Capital: $1182323.14
      --> Done. Writing CSV file
Getting tickers for 2002-03-17
   Calculating 
100%
   Starting Capital: $1182323.14
      Profit from options: $9053.0
      Profit from interest: $2043.45
   Ending Capital: $1193419.59
      --> Done. Writing CSV file
Getting tickers for 2002-04-21
   Calculating 
100%
   Starting Capital: $1193419.59
      Profit from options: $5233.5
      Profit from interest: $1512.66
   Ending Capital: $1200165.75
      --> Done. Writing CSV file
Getting tickers for 2002-05-19
   Calculating 
100%
   Starting Capital: $1200165.75
      Profit from options: $-3793.6
      Profit from interest: $1949.6
   Ending Capital: $1198321.75
      --> Done. Writing CSV file
Getting tickers for 2002-06-23
   Calculating 
100%
   Starting Capital: $1198321.75
      Profit from options: $-56830.3
      Profit from interest: $1527.86
   E

   Calculating 
100%
   Starting Capital: $1426996.39
      Profit from options: $16220.7
      Profit from interest: $3841.0
   Ending Capital: $1447058.09
      --> Done. Writing CSV file
Getting tickers for 2005-05-22
   Calculating 
100%
   Starting Capital: $1447058.09
      Profit from options: $10368.7
      Profit from interest: $3158.2
   Ending Capital: $1460584.99
      --> Done. Writing CSV file
Getting tickers for 2005-06-19
   Calculating 
100%
   Starting Capital: $1460584.99
      Profit from options: $8232.5
      Profit from interest: $3242.5
   Ending Capital: $1472059.99
      --> Done. Writing CSV file
Getting tickers for 2005-07-17
   Calculating 
100%
   Starting Capital: $1472059.99
      Profit from options: $-20730.6
      Profit from interest: $4490.6
   Ending Capital: $1455819.99
      --> Done. Writing CSV file
Getting tickers for 2005-08-21
   Calculating 
100%
   Starting Capital: $1455819.99
      Profit from options: $12540.0
      Profit from interest

   Calculating 
100%
   Starting Capital: $1731704.61
      Profit from options: $-29590.7
      Profit from interest: $2927.54
   Ending Capital: $1705041.45
      --> Done. Writing CSV file
Getting tickers for 2008-06-22
   Calculating 
100%
   Starting Capital: $1705041.45
      Profit from options: $5765.2
      Profit from interest: $2365.75
   Ending Capital: $1713172.4
      --> Done. Writing CSV file
Getting tickers for 2008-07-20
   Calculating 
100%
   Starting Capital: $1713172.4
      Profit from options: $2062.4
      Profit from interest: $1945.4
   Ending Capital: $1717180.2
      --> Done. Writing CSV file
Getting tickers for 2008-08-18
   Calculating 
100%
   Starting Capital: $1717180.2
      Profit from options: $-27907.2
      Profit from interest: $2817.61
   Ending Capital: $1692090.61
      --> Done. Writing CSV file
Getting tickers for 2008-09-21
   Calculating 
100%
   Starting Capital: $1692090.61
      Profit from options: $-351191.0
      Profit from interes

ProgrammingError: (psycopg2.ProgrammingError) permission denied for relation secm
 [SQL: "select gvkey,cyear,tpci,exchg,datadate from compm.secm where datadate > '2010-06-30' and datadate <= '2010-09-30' and tpci in ('0','F') and exchg in (11,12,14,19) "] (Background on this error at: http://sqlalche.me/e/f405)